In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 1.6 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


#import thư viện

In [ ]:
from sklearn.svm import SVR# mô hình chạy thử
from sklearn.svm import SVC# mô hình chạy thử
from sklearn.neighbors import KNeighborsClassifier# mô hình chạy thử
import pandas as pd# lấy dữ liệu
import numpy as np # tính toán
import matplotlib.pyplot as plt # vẽ đồ thị
from sklearn.model_selection import train_test_split # Chia dữ liệu
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix #đánh giá mô hình
import re # tiền xử lý
import string # tiền xử lý
from sklearn.decomposition import PCA # giảm chiều dữ liệu
from sklearn.feature_extraction.text import TfidfVectorizer # Tính tfidf
import joblib # lưu model
from joblib import load as ld
from sklearn.preprocessing import StandardScaler

#Hàm

In [ ]:
vietnamese_stop_words = [
              'là', 'và', 'có', 'trên', 'cho', 'một', 'các', 'được', 'như', 'với',
              'của', 'ở', 'khi', 'đã', 'còn', 'thì', 'này', 'bởi', 'đó', 'để', 'năm',
              'ngày', 'vào', 'sau', 'tại', 'trong', 'rằng', 'đến', 'từ'
              ]

In [ ]:
# model = SentenceTransformer('/content/drive/MyDrive/SIC/model_doc2vec')

In [ ]:
#Hàm làm sạch dữ liệu <nên dùng trước khi vector hoá dữ liệu>
def convertData(text):
  text = text.lower()
  text = re.sub('\[.*?\]','',text)
  text = re.sub("\\W"," ",text)
  text = re.sub('https?://\S+|www\.\S+','',text)
  text = re.sub('<.*?>+',b'',text)
  text = re.sub('[%s]' % re.escape(string.punctuation),'',text)
  text = re.sub('\w*\d\w*','',text)
  text = re.sub(' +', ' ',text)
  text = text.strip()
  text = ' '.join([word for word in text.split() if word not in vietnamese_stop_words])
  return text

<h2>Cosine_similarity

In [ ]:
#Hàm để tính Cosine
def cosine_similarity(x_vec,y_vec, flag):
  cosine_similarities = x_vec.dot(y_vec.T)
  if flag == True:# Trường hợp trả kết quả với dữ liệu tfIdf
    return cosine_similarities.toarray().squeeze()#
  else:# Trường hợp trả kết quả với dữ liệu ebd
    ll = []
    for i in range(cosine_similarities.shape[0]):
      ll.append(cosine_similarities[i].max())
    return ll

In [ ]:
#Hàm kiểm thử cho đánh giá độ tương đồng Cosine
def Cosine_sml(fake_news,real_vectors):
  # lấy chỉ số của từng fake_news
  content = [convertData(txt) for txt in fake_news['Fake_Content']]# làm sạch từng bài
  # vector hoá từng bài
  vectors_content = vectorizer.transform(content)
  cosine_similarities = cosine_similarity(vectors_content,real_vectors, True) # gọi hàm để tính Cosine
  danhgia_f = 0
  for idx, x in enumerate(cosine_similarities):
    if x.max() < 0.5:
      danhgia_f +=1
  print(f'Tổng số tin giả của fake_news : {len(fake_news)}')
  print(f'Số lượng tin được phát hiện có khả năng là giả của fake_news: {danhgia_f}')
  print(f"Độ chính xác khi dự đoán tin giả : {danhgia_f/len(fake_news)}")
  print('_____________________________________________________________________________________')
  return danhgia_f/len(fake_news)

In [ ]:
#Hàm kiểm thử cho đánh giá độ tương đồng Cosine
def Cosine_sml_ebd(fake_vectors,real_vectors):#fake_vectors va real_vectors
  cosine_similarities = cosine_similarity(fake_vectors,real_vectors, False)
  danhgia_f = 0
  lt = list()
  for idx, x in enumerate(cosine_similarities):
    lt.append(x.max())
  scalling = StandardScaler()
  lt = scalling.fit_transform(np.array(lt).reshape(-1,1))
  for idx, x in enumerate(lt):
    if x.max() < 0.5:
      danhgia_f +=1
  print(f'Tổng số tin giả của fake_ : {len(fake_vectors)}')
  print(f'Số lượng tin được phát hiện có khả năng là giả của fake_: {danhgia_f}')
  print(f"Độ chính xác khi dự đoán tin giả : {danhgia_f/len(fake_vectors)}")
  print('_____________________________________________________________________________________')
  return danhgia_f/len(fake_vectors)

#Gọi dữ liệu test

In [ ]:
#real_news là phần tập dữ liệu gốc (8943 bài viết-tin thật) chưa convert và chưa vector hoá
# 2 tập này chưa xử lý gì hết
real_news = pd.read_csv('/content/drive/MyDrive/Data_/Data_test_300/DATA_.csv')#Tập tin có thể là thật
teptingia = pd.read_csv('/content/drive/MyDrive/Data_/dulieutin_gia_4000.csv')#Tập tin có thể là giả
#2 tập ebd này đã được dùng bert để vector hoá
ebd_teptingia = pd.read_csv('/content/drive/MyDrive/Data_/Fake_Ebd_4000.csv') # 4000 dữ liệu đầu tiên
ebd_teptinthat = pd.read_csv('/content/drive/MyDrive/SIC/Embeddings_DATA.csv')
real_news.drop(columns=['Unnamed: 0'],inplace=True)
# real_news.columns = ['Content']

real_news = real_news[4000:]
real_news.reset_index(inplace=True)
real_news.drop(columns=['index'],inplace=True)
teptingia.drop(columns=['Unnamed: 0'],inplace=True)
teptingia.reset_index(inplace=True)
teptingia.drop(columns=['Unnamed: 0.1'],inplace=True)
teptingia.drop(columns=['index'],inplace=True)
ebd_teptinthat.drop(columns=['Unnamed: 0'],inplace=True)
ebd_teptingia.drop(columns=['Unnamed: 0'],inplace=True)
ebd_teptinthat = ebd_teptinthat[4000:]
ebd_teptinthat.reset_index(inplace=True)
ebd_teptinthat.drop(columns=['index'],inplace=True)

<ipython-input-12-fd2884e2c613>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real_news.drop(columns=['index'],inplace=True)
<ipython-input-12-fd2884e2c613>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ebd_teptinthat.drop(columns=['index'],inplace=True)


In [ ]:
ketqua_tfidf = list()
ketqua_ebd = list()

In [ ]:
acr = list()
prec = list()
f1 = list()

#Độ tương đồng Cosine

<h1>TfIDF</h1>

In [ ]:
vectorizer = TfidfVectorizer()
real_vectors = vectorizer.fit_transform(real_news['Content'])

In [ ]:
ketqua_tfidf.append(Cosine_sml(teptingia,real_vectors))#tfidf

Tổng số tin giả của fake_news : 4000
Số lượng tin được phát hiện có khả năng là giả của fake_news: 2170
Độ chính xác khi dự đoán tin giả : 0.5425
_____________________________________________________________________________________


<h1>Bert</h1>

In [ ]:
ketqua_ebd.append(Cosine_sml_ebd(ebd_teptingia,ebd_teptinthat))

Tổng số tin giả của fake_ : 4000
Số lượng tin được phát hiện có khả năng là giả của fake_: 2828
Độ chính xác khi dự đoán tin giả : 0.707
_____________________________________________________________________________________


## SVC - KNN

#Sử dụng tfIdf để vector hoá

SVM_SVC

In [ ]:
#gọi model
svc_model = joblib.load('/content/drive/MyDrive/SIC/svc_model_update.pkl')

In [ ]:
#Làm sạch
X_Fake_tfidf = [convertData(teptingia['Fake_Content'][i]) for i in range(len(teptingia['Fake_Content']))]
X_Real_tfidf = [convertData(real_news['Content'][i]) for i in range(len(real_news['Content']))]
# Tổng hợp và đánh nhãn
X = X_Real_tfidf + X_Fake_tfidf
Y = [1]*len(X_Real_tfidf) + [0]*len(X_Fake_tfidf)
#Chia dữ liêu train và test : 80 20
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
vectorizer = joblib.load('/content/drive/MyDrive/SIC/vectorizer_tfidf.joblib')
X_train_tfidf = vectorizer.transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
y_pred = svc_model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
ketqua_tfidf.append(accuracy)
print("Accuracy: {:.2f}%".format(accuracy * 100))
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 78.42%
Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.79      0.77       819
           1       0.82      0.78      0.80       970

    accuracy                           0.78      1789
   macro avg       0.78      0.79      0.78      1789
weighted avg       0.79      0.78      0.78      1789

Confusion Matrix:
[[651 168]
 [218 752]]


KNN

In [ ]:
knn_model = ld('/content/drive/MyDrive/SIC/knn_model_Update.joblib')

In [ ]:
y_pred = knn_model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
ketqua_tfidf.append(accuracy)
print("Accuracy: {:.2f}%".format(accuracy * 100))
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

KeyboardInterrupt: 

#Sử dụng bert để vector hoá

In [ ]:
svc_model_ebd = joblib.load('/content/drive/MyDrive/SIC/svc_model_ebd.pkl')

In [ ]:
X_bert = pd.concat([ebd_teptinthat,ebd_teptingia])
y_bert = [1]*len(ebd_teptinthat) + [0]*len(ebd_teptingia)
X_bert.reset_index(inplace=True)
X_bert.drop(columns=['index'],inplace=True)

In [ ]:
scalling = StandardScaler()
X_bert = scalling.fit_transform(X_bert)

In [ ]:
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(X_bert, y_bert, test_size=0.2, random_state=42)

In [ ]:
y_pred = svc_model_ebd.predict(X_test_b)
accuracy = accuracy_score(y_test_b, y_pred)
ketqua_ebd.append(accuracy)
print("Accuracy: {:.2f}%".format(accuracy * 100))
print("Classification Report:")
print(classification_report(y_test_b, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test_b, y_pred))

<h2>knn</h2>

In [ ]:
knn_model_ebd = joblib.load('/content/drive/MyDrive/SIC/knn_model_ebd.pkl')

In [ ]:
y_pred = knn_model_ebd.predict(X_test_b)
accuracy = accuracy_score(y_test_b, y_pred)
ketqua_ebd.append(accuracy)
print("Accuracy: {:.2f}%".format(accuracy * 100))
print("Classification Report:")
print(classification_report(y_test_b, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test_b, y_pred))

#Vẽ kết quả

In [ ]:
#ketqua_tfidf, ketqua_ebd
#SVC, KNN
plt.figure(figsize=(12,8))
plt.bar(x='SVC', ketqua_tfidf[0], width=0.3, label='SVC')
plt.bar(x='KNN', ketqua_tfidf[1], width=0.3, label='KNN')
plt.ylim(0,1)
plt.legend(['SVC','KNN'])
plt.show()